## Part 2a: generate disv properties

In [ ]:
import sys, json, os
import rasterio, flopy
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from mf6Voronoi.meshProperties import meshShape
from shapely.geometry import MultiLineString

In [ ]:
# open the json file
with open('json/disvDict.json') as file:                        #<============ Check this ============
    gridProps = json.load(file)

In [ ]:
cell2d = gridProps['cell2d']           #cellid, cell centroid xy, vertex number and vertex id list
vertices = gridProps['vertices']       #vertex id and xy coordinates
ncpl = gridProps['ncpl']               #number of cells per layer
nvert = gridProps['nvert']             #number of verts
centroids=gridProps['centroids']       #cell centroids xy 

## Part 2b: Model construction and simulation

In [ ]:
#Extract dem values for each centroid of the voronois
src = rasterio.open('rst/asterDem18S.tif')                     #<============ Check this ============
elevation=[x for x in src.sample(centroids)]

In [ ]:
nlay = 5                                                      #<============ Check this ============

mtop=np.array([elev[0] for i,elev in enumerate(elevation)])
zbot=np.zeros((nlay,ncpl))


AcuifInf_Bottom = 2800
zbot[0,] = mtop - 30
zbot[1,] = AcuifInf_Bottom + (0.85 * (mtop - AcuifInf_Bottom))
zbot[2,] = AcuifInf_Bottom + (0.70 * (mtop - AcuifInf_Bottom))
zbot[3,] = AcuifInf_Bottom + (0.50 * (mtop - AcuifInf_Bottom))
zbot[4,] = AcuifInf_Bottom

### Create simulation and model

In [ ]:
# create simulation
simName = 'mf6Sim'
modelName = 'mf6Model'
modelWs = 'modelFiles'
sim = flopy.mf6.MFSimulation(sim_name=modelName, version='mf6', 
                             exe_name='bin/mf6.exe', 
                             sim_ws=modelWs)

In [ ]:
# create tdis package
tdis_rc = [(1000.0, 1, 1.0)]
tdis = flopy.mf6.ModflowTdis(sim, pname='tdis', time_units='DAYS', 
                             perioddata=tdis_rc)

In [ ]:
# create gwf model
gwf = flopy.mf6.ModflowGwf(sim, 
                           modelname=modelName, 
                           save_flows=True,
                           newtonoptions="NEWTON UNDER_RELAXATION")

In [ ]:
# create iterative model solution and register the gwf model with it
ims = flopy.mf6.ModflowIms(sim,
                           complexity='COMPLEX',
                           outer_maximum=50,
                           inner_maximum=30, 
                           linear_acceleration='BICGSTAB')
sim.register_ims_package(ims,[modelName])

In [ ]:
# disv
disv = flopy.mf6.ModflowGwfdisv(gwf, nlay=nlay, ncpl=ncpl, 
                                top=mtop, botm=zbot, 
                                nvert=nvert, vertices=vertices, 
                                cell2d=cell2d)

In [ ]:
# initial conditions
ic = flopy.mf6.ModflowGwfic(gwf, strt=np.stack([mtop for i in range(nlay)]))

In [ ]:
Kx =[4E-4,5E-6,1E-6,9E-7,5E-7]                                    #<============ Check this ============
icelltype = [1,1,0,0,0]

# node property flow
npf = flopy.mf6.ModflowGwfnpf(gwf, xt3doptions=[('xt3d')],
                              save_specific_discharge=True,
                              icelltype=icelltype, 
                              k=Kx)

In [ ]:
# define storage and transient stress periods
sto = flopy.mf6.ModflowGwfsto(gwf,
                              iconvert=1,
                              steady_state={
                                0:True,
                              }
                              )

#### Working with rechage, evapotranspiration

In [ ]:
rchr = 0.15/365/86400                                          #<============ Check this ============
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=rchr)
evtr = 1.2/365/86400                                           #<============ Check this ============
evt = flopy.mf6.ModflowGwfevta(gwf,ievt=1,surface=mtop,rate=evtr,depth=1.0)

### Definition of the intersect object
For the manipulation of spatial data to determine hydraulic parameters or boundary conditions

In [ ]:
# Define intersection object
interIx = flopy.utils.gridintersect.GridIntersect(gwf.modelgrid)

In [ ]:
#open the river shapefile
rivers =gpd.read_file('shp/river_basin.shp')                   #<============ Check this ============
list_rivers=[]
for i in range(rivers.shape[0]):
    list_rivers.append(rivers['geometry'].loc[i])
    
riverMls = MultiLineString(lines=list_rivers)

#intersec rivers with our grid
riverCells=interIx.intersect(riverMls).cellids
riverCells[:10]

In [ ]:
#river package
riverSpd = {}
riverSpd[0] = []
for cell in riverCells:
    riverSpd[0].append([(0,cell),mtop[cell],0.01]) 
riv = flopy.mf6.ModflowGwfdrn(gwf, stress_period_data=riverSpd)

In [ ]:
#river plot
riv.plot(mflay=0)

### Set the Output Control and run simulation

In [ ]:
#oc
head_filerecord = f"{gwf.name}.hds"
oc = flopy.mf6.ModflowGwfoc(gwf,
                            head_filerecord=head_filerecord,
                            saverecord=[("HEAD", "LAST")])

In [ ]:
# Run the simulation
sim.write_simulation()
success, buff = sim.run_simulation()

### Model output visualization

In [ ]:
headObj = gwf.output.head()
headObj.get_kstpkper()

In [ ]:
heads = headObj.get_data()
heads[2,0,:5]

In [ ]:
# Plot the heads for a defined layer and boundary conditions
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
modelmap = flopy.plot.PlotMapView(model=gwf)

####
levels = np.linspace(heads[heads>-1e+30].min(),heads[heads>-1e+30].max(),num=50)
contour = modelmap.contour_array(heads[3],ax=ax,levels=levels,cmap='PuBu')
ax.clabel(contour)


quadmesh = modelmap.plot_bc('DRN')
cellhead = modelmap.plot_array(heads[3],ax=ax, cmap='Blues', alpha=0.8)

linecollection = modelmap.plot_grid(linewidth=0.3, alpha=0.5, color='cyan', ax=ax)

plt.colorbar(cellhead, shrink=0.75)

plt.show()